In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [15]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [16]:
def clean_reviews(no_reviews,dataset):
    import re
    import nltk
    nltk.download('stopwords') 
    from nltk.corpus import stopwords 
    
    my_stopwords = stopwords.words('english') 
            
    from nltk.stem.porter import PorterStemmer
     
    corpus =[] 
    if no_reviews == 35000:
        start = 0
        end = 35000
    else:
        start = 35000
        end = 50000
    for i in range (start, end):
        review = re.sub('[^a-zA-Z]',' ', dataset[i]) #Removes all other characters besides the letters and puts the review in a string
        review = review.lower() 
        review = review.split()
        ps = PorterStemmer() 
        review = [ps.stem(word) for word in review if not word in set (my_stopwords)]
        review = ' '.join(review) 
        corpus.append(review)
    return corpus

In [17]:
train = df.loc[0:34999]
test = df.loc[35000:]
train_data = train['review']
test_data = test['review']
train_labels = train['label']
test_labels = test['label']

In [18]:
train_data = clean_reviews(35000,train_data)
test_data = clean_reviews(50000,test_data)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
train_labels = train_labels.map({'pos': 1, 'neg': 0})
test_labels = test_labels.map({'pos': 1, 'neg': 0})

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
train_data = vect.fit_transform(train_data)

In [21]:
test_data = vect.transform(test_data)

In [22]:
from sklearn.linear_model import LogisticRegression
nb = LogisticRegression()

In [23]:
nb.fit(train_data, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
y_pred_class = nb.predict(test_data)

In [25]:
from sklearn import metrics
metrics.accuracy_score(test_labels, y_pred_class)

0.8898666666666667

In [26]:
metrics.confusion_matrix(test_labels, y_pred_class)

array([[6606,  918],
       [ 734, 6742]], dtype=int64)